In [5]:
import numpy as np
import bz2
from functools import partial
from collections import Counter
import pickle
from itertools import islice
from xml.etree import ElementTree
import codecs
import csv
import time
import os
import re
from pathlib import Path
from google.cloud import storage


In [ ]:
### page views###

# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
print(p)

pv_name = p.name
print(pv_name)
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
print("download done")
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
print("done all")
# write out the counter as binary file (pickle it)
# with open(pv_clean, 'wb') as f:
#   pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

https:/dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
pageviews-202108-user.bz2
--2022-01-08 22:50:20--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.65MB/s    in 8m 41s  

2022-01-08 22:59:01 (4.59 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]

download done
done all


In [3]:
file_name = 'page_view_dict'
base_dir = '.'
with open(Path(base_dir) / f'{file_name}.pkl','wb') as f:
	pickle.dump(wid2pv,f)

In [7]:
###reading from the bucket:
client = storage.Client()
page_view_dict = None
for blob in client.list_blobs('ass-3-bucket-tamar', prefix='page_view_dict'):
    if blob.name.endswith("pkl"):
        pickle_in = blob.download_as_string()
        page_view_dict = pickle.loads(pickle_in)

In [8]:
# read in the counter


def get_pageview(wiki_ids):
    page_views =[]  
    for wiki_id in wiki_ids:
      page_views.append(page_view_dict[wiki_id])
    return page_views

In [11]:
wiki_ids = [4105482,143377,72573,31496,31034,11313,30475]
print(get_pageview(wiki_ids))

[715, 513629, 699, 60468, 31420, 26162, 79286]
